In [3]:
import torch
import torchvision
import tensorflow as tf
from torchvision import models, datasets
print(tf.__version__)

import numpy as np
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.linear_model import LogisticRegression
BATCH_SIZE = 32
LEARNING_RATE = 0.1
EPOCHS = 100
from keras.datasets import  cifar10
print(tf.__version__)

# set transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
    )])
# initialize model that outputs fc6 layer
alexnetfc6 = models.alexnet(pretrained=True)
fc6Clacifier = torch.nn.Sequential(*list(alexnetfc6.classifier.children())[:-2])
alexnetfc6.classifier = fc6Clacifier

#import data
trainData = datasets.CIFAR10(root = '.\data',train=True, download=True, transform=transform)
testData = datasets.CIFAR10(root = '.\data',train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testData, shuffle=False)
trainloader = torch.utils.data.DataLoader(trainData, shuffle=True)
#make train data into tensor
x_train = trainData.data
y_train = trainData.targets
x_train = np.array(x_train)
y_train = np.array(y_train)
x_train = torch.from_numpy(x_train )
y_train = torch.from_numpy(y_train )
#make test data into tensor and seperate into val and test sets
x_test = testData.data
y_test = testData.targets
x_test = np.array(x_test)
y_test = np.array(y_test)
x_test = torch.from_numpy(x_test )
y_test = torch.from_numpy(y_test )
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=1)
#create datasets
dataset = torch.utils.data.TensorDataset(x_train, y_train)
dataset = torch.utils.data.DataLoader(dataset, BATCH_SIZE)
val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
val_dataset = torch.utils.data.DataLoader(val_dataset)
test_dataset = torch.utils.data.TensorDataset(x_test, y_test)
test_dataset = torch.utils.data.DataLoader(test_dataset)
with torch.no_grad():
    #put through model and get training image features
    featuresDataset =  torch.tensor([[]])
    count = 0
    trainLabels = np.zeros(len(trainloader))
    firstTime = True
    for batch in trainloader:
        images, label = batch
        trainLabels[count] = label
        alexnetfc6.eval()
        output = alexnetfc6(images)
        if(firstTime == True):
            featuresDataset = output
            firstTime = False
        else:
            featuresDataset = torch.cat((featuresDataset, output),dim=0)
        count+=1

    featuresDatasetTrain =  featuresDataset.detach().numpy()

    #put through model and get testing image features
    featuresDataset =  torch.tensor([])
    count = 0
    testLabels = np.zeros(len(testloader))
    firstTime = True
    for batch in testloader:
            images, label = batch
            testLabels[count] = label
            alexnetfc6.eval()
            output = alexnetfc6(images)
            if(firstTime == True):
                featuresDataset = output
                firstTime = False
            else:
                featuresDataset = torch.cat((featuresDataset, output),dim=0)
            count+=1
    featuresDatasetTest =  featuresDataset.detach().numpy()


    #set up logistic regression
    logReg = LogisticRegression(max_iter=1000)
    logReg.fit(featuresDatasetTrain,trainLabels)

    #predict with  logistic regression and count correct predictions
    correctPredictions = 0
    count = 0
    for features in featuresDatasetTest:
        prediction = logReg.predict(features.reshape(1,-1))
        if (prediction == testLabels[count]):
            correctPredictions += 1
        count +=1
score = logReg.score(featuresDatasetTest[0:len(testloader)], testLabels[0:len(testloader)])
print("Logistic regresion trained with alexnetfc6 layer data")
print("Total predictions:" + str(len(testloader)))
print("Correct predictions:" + str(correctPredictions))
print("Acuracy:" + str(score*100) + "%")

2.4.1
2.4.1
Files already downloaded and verified
Files already downloaded and verified


/home/cameron/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic regresion trained with alexnetfc6 layer data
Total predictions:10000
Correct predictions:8239
Acuracy:82.39%
